# AUTHORITATIVE VERSION AS OF 2023/09/14

This notebook fits the NP params to Zea lettuce data
* mN and YN are constrained to empirical values
* hybrid MEC-NP model plot is generated

## Import libraries and globally set parameters for plotting

In [ ]:
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator

from DU4 import *
import time

import matplotlib.style as style
plt.rcParams.update(plt.rcParamsDefault)
style.use('seaborn-poster')
plt.rcParams['figure.dpi'] = 300

'''
Spines & lines
'''
box_lw = 1
mono_colr = 'k'
plt.rcParams['axes.spines.bottom'] = True
plt.rcParams['axes.spines.left'] = True
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.linewidth'] = box_lw
plt.rcParams['xtick.major.width'] = box_lw
plt.rcParams['ytick.major.width'] = box_lw
'''
Fonts & size
'''
plt_font_size = 8
lgd_font_size = 5
plt.rcParams['font.family'] = "TeX Gyre Termes"
#plt.rc('font', **{'family' : 'sans-serif', 'sans-serif' : ['Myriad Pro']})
plt.rcParams['font.size'] = plt_font_size
plt.rcParams['axes.labelsize'] = plt_font_size
plt.rcParams['axes.titlesize'] = plt_font_size
plt.rcParams['xtick.labelsize'] = plt_font_size
plt.rcParams['ytick.labelsize'] = plt_font_size
plt.rcParams['xtick.major.pad'] = 2
plt.rcParams['ytick.major.pad'] = 2
plt.rcParams['legend.fontsize'] = lgd_font_size
'''
Plots
'''
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['lines.markeredgewidth'] = 3
plt.rcParams['errorbar.capsize'] = 5
plt.rcParams['lines.markersize'] = 4
'''
Colours
'''
plt.rcParams['axes.titlecolor'] = mono_colr
plt.rcParams['axes.edgecolor'] = mono_colr
plt.rcParams['axes.labelcolor'] = mono_colr
plt.rcParams['xtick.color'] = mono_colr
plt.rcParams['xtick.labelcolor'] = mono_colr
plt.rcParams['ytick.color'] = mono_colr
plt.rcParams['ytick.labelcolor'] = mono_colr
'''
LaTeX
'''
plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.preamble'] = '\n'.join([
    r'\usepackage[T3,T1]{fontenc}',
    r'\DeclareSymbolFont{tipa}{T3}{cmr}{m}{n}',
    r'\DeclareMathAccent{\invbreve}{\mathalpha}{tipa}{16}',
    r'\usepackage{siunitx}',
    r'\DeclareSIUnit\crewmember{CM}',
    r'\sisetup{range-units=single}',
    r'\sisetup{range-phrase=\textup{--}}'
])  # Preamble must be one line!

## Set global variables

In [ ]:
hg_color="MidnightBlue"
md_color="DodgerBlue"
lw_color="SkyBlue"
np_color="k"

out_path = "./NP-fits/"
mat_out_path = "./sens-mat/"

c_CO2 = 525
Phi_gamma = 225
sigma_N = 19.2 # planting density
f_E = 1 # we look at the whole plant, not just edible part
endtime = 50

## Define functions for models

In [ ]:
def run_models_less(params, exp_times=None):
    
    #print(params.shape)
    def calc_eta_u(t, crop, c_CO2, Phi_gamma):
        '''
        N uptake performance, 1 = max
        Could account for things like temp, pH
        Assume max for now
        Dimensionless
        '''
        return 1

    def calc_mu_N(t, crop, c_CO2, Phi_gamma):
        '''
        seems to decrease over time
        needs to depend on c_N where high or low c_N limits mu_N
        could measure by mu_N = [ln(m_N(t2)) - ln(m_N(t1))] / (t2 - t1)
        linear fit from Normal N
        units of day^-1
        '''
        mu_N = -params[6] * t + params[5]
        return mu_N

    def calc_eta_N(t, crop, c_CO2, Phi_gamma):
        '''
        amount of plant you get per amount of N over time step
        eta_N = m_B / <m_N>
        dimensionless, but g_DW / g_N
        '''
        eta_N = -params[4]*t + params[3] 
        return eta_N

    def calc_m_N(t, crop, c_CO2, Phi_gamma):
        '''
        m_N0 unit: g
        K unit: g
        r unit: day^-1
        m_N unit: g
        '''
        m_N0 = 5.1e-3  #Got it from fitting with MEC, estimate from data using N percentage in biomass
        r = params[0]
        K = params[1]
        alpha = params[2]
        m_N = alpha * (m_N0 * K * np.exp(r * t)) / ((K - m_N0) + m_N0 * np.exp(r * t))
        return m_N

    def calc_Y_N(t, crop, c_CO2, Phi_gamma):
        '''
        Calculated value is too high vs. empirical data - should be ~2 from day 23-37
        Seems to be because eta_N is too high early on
        '''
        Y_N = calc_eta_u(t, crop, c_CO2, Phi_gamma) * calc_mu_N(t, crop, c_CO2, Phi_gamma) * calc_eta_N(t, crop, c_CO2, Phi_gamma)
        return Y_N

    def calc_m_B_NP(t, crop, c_CO2, Phi_gamma):
        return calc_Y_N(t,crop,c_CO2,Phi_gamma) * calc_m_N(t,crop,c_CO2,Phi_gamma)

    def NP_model(t, y, crop, c_CO2, Phi_gamma):
        Neq = len(y)

        ## Prepare dydt array
        dydt = np.zeros((1, Neq))

        ## Define dydt
        dydt[0, 0] = calc_m_B_NP(t, crop, c_CO2, Phi_gamma)

        return [np.transpose(dydt)]

    # for all models
    tspan = [0, endtime]
    if len(exp_times) == 0:
        t_eval = np.arange(0, endtime+1, 1) #Where do we want the solution
    else:
        t_eval = exp_times.reshape(-1,)
    y0 = [0,0,50]

    sol_NP = integrate.solve_ivp(NP_model, tspan, y0, args=("lettuce",c_CO2,Phi_gamma), method='LSODA', t_eval=t_eval)
    sol_NP.y[0] = sol_NP.y[0] * sigma_N * f_E

    # fig, ax1 = plt.subplots(1, figsize=(5, 5))
    # plt.ylim(0,120)
    # plt.plot(sol_NP.t, sol_NP.y[0],linewidth=4, color = 'blue', label="NP normal")
    # plt.ylabel('Edible Biomass $[\si{\gram\of{DW}\per\meter\squared}]$')
    # plt.xlabel('Time, t [d$_{\mathrm{AE}}$]')
    # plt.title('NP Model of Biomass Growth')

    # Put MEC on top of it
    def mec_model(t, y, crop, CO2, PPF):
        Neq = len(y)

        ## Prepare dydt array
        dydt = np.zeros((1, Neq))

        ## Define dydt
        dydt[0, 0] = calc_m_B(t, crop, CO2, PPF)

        return [np.transpose(dydt)]

    ## Define directory and locations
    directory = 'parameter-lists/'
    filename = 'crop_parameters_FPSD.xlsx'

    ## Load standard parameters from BVAD
    filename_full = directory + filename

    ## Load the standard lettuce crop
    crop_type = 'lettuce'
    crop = Crop(crop_type, filename_full=filename_full)
    crop.t_M = endtime

    ## Perform integration
    start_time = time.time()
    sol_MEC = integrate.solve_ivp(mec_model, tspan, y0, args=(crop,c_CO2,Phi_gamma), method='LSODA', t_eval=t_eval)
   # print("--- %s seconds ---" % (time.time() - start_time))

    ## Convert m_T to m_E (with lettuce t_E = 0)
    sol_MEC.y[0] *= crop.f_E

    #plt.plot(sol_MEC.t, sol_MEC.y[0], linewidth=4, color = 'g', ls='--', label="MEC prediction")
    #plt.legend()
    #plt.show()

    # show function values over time
    f=np.zeros((50,1))
    g=np.zeros((50,1))
    h=np.zeros((50,1))
    j=np.zeros((50,1))

    for i in range(0,50):
        f[i] = calc_mu_N(i, crop, c_CO2, Phi_gamma)
        g[i] = calc_m_N(i, crop, c_CO2, Phi_gamma)
        h[i] = calc_Y_N(i, crop, c_CO2, Phi_gamma)
        j[i] = calc_eta_N(i, crop, c_CO2, Phi_gamma)

    #plt.plot(np.arange(0,50), f, label="$\mu_N$")
    #plt.plot(np.arange(0,50), g, label="$m_N$")
    #plt.plot(np.arange(0,50), h, label="$\dot{Y}_N$")
    #plt.plot(np.arange(0,50), j, label="$\eta_N$")
    #plt.legend()
    # Create arrays to store m_N and Y_N values for sol_NP.t
    m_N_values = np.zeros(len(sol_NP.t))
    Y_N_values = np.zeros(len(sol_NP.t))

    # Evaluate m_N and Y_N for each time point in sol_NP.t
    for i, t in enumerate(sol_NP.t):
        m_N_values[i] = calc_m_N(t, crop, c_CO2, Phi_gamma)
        Y_N_values[i] = calc_Y_N(t, crop, c_CO2, Phi_gamma)

    # Return the arrays along with the other return values
    return sol_MEC.t, sol_MEC.y[0], sol_NP.t, sol_NP.y[0], m_N_values, Y_N_values

In [ ]:
def run_models(params, exp_times=None):
    
    #print(params.shape)
    def calc_eta_u(t, crop, c_CO2, Phi_gamma):
        '''
        N uptake performance, 1 = max
        Could account for things like temp, pH
        Assume max for now
        Dimensionless
        '''
        return 1

    def calc_mu_N(t, crop, c_CO2, Phi_gamma):
        '''
        seems to decrease over time
        needs to depend on c_N where high or low c_N limits mu_N
        could measure by mu_N = [ln(m_N(t2)) - ln(m_N(t1))] / (t2 - t1)
        linear fit from Normal N
        units of day^-1
        '''
        mu_N = -params[7] * t + params[6]
        return mu_N

    def calc_eta_N(t, crop, c_CO2, Phi_gamma):
        '''
        amount of plant you get per amount of N over time step
        eta_N = m_B / <m_N>
        can approach zero, but should not become negative
        *** FIX THIS ***
        dimensionless, but g_DW / g_N
        '''
        eta_N = -params[5]*t + params[4] 
        return eta_N

    def calc_m_N(t, crop, c_CO2, Phi_gamma):
        '''
        Maybe Monod-style kinetics as GM suggested

        4 parameter logistic fit?

        All values except m_N0 are guesses not based on data

        The value of m_N is too high vs. empirical data
            * Empirical data could be sigmoidal
            * Maybe we call it something else

        m_N0 unit: g
        K unit: g
        r unit: day^-1
        m_N unit: g
        '''
        m_N0 = params[0]
        r = params[1]
        K = params[2]
        alpha = params[3]
        m_N = alpha * (m_N0 * K * np.exp(r * t)) / ((K - m_N0) + m_N0 * np.exp(r * t))  # huger guess
        return m_N

    def calc_Y_N(t, crop, c_CO2, Phi_gamma):
        '''
        Calculated value is too high vs. empirical data - should be ~2 from day 23-37
        Seems to be because eta_N is too high early on
        '''
        Y_N = calc_eta_u(t, crop, c_CO2, Phi_gamma) * calc_mu_N(t, crop, c_CO2, Phi_gamma) * calc_eta_N(t, crop, c_CO2, Phi_gamma)
        return Y_N

    def calc_m_B_NP(t, crop, c_CO2, Phi_gamma):
        return calc_Y_N(t,crop,c_CO2,Phi_gamma) * calc_m_N(t,crop,c_CO2,Phi_gamma)

    def NP_model(t, y, crop, c_CO2, Phi_gamma):
        Neq = len(y)

        ## Prepare dydt array
        dydt = np.zeros((1, Neq))

        ## Define dydt
        dydt[0, 0] = calc_m_B_NP(t, crop, c_CO2, Phi_gamma)

        return [np.transpose(dydt)]
        
    # for all models
    tspan = [0, endtime]
    if len(exp_times) == 0:
        t_eval = np.arange(0, endtime+1, 1) #Where do we want the solution
    else:
        t_eval = exp_times.reshape(-1,)
    y0 = [0,0,50]

    sol_NP = integrate.solve_ivp(NP_model, tspan, y0, args=("lettuce",c_CO2,Phi_gamma), method='LSODA', t_eval=t_eval)
    sol_NP.y[0] = sol_NP.y[0] * sigma_N * f_E

    #fig, ax1 = plt.subplots(1, figsize=(5, 5))
    # plt.ylim(0,120)
    #plt.plot(sol_NP.t, sol_NP.y[0],linewidth=4, color = 'blue', label="NP normal")
    #plt.ylabel('Edible Biomass $[\si{\gram\of{DW}\per\meter\squared}]$')
    #plt.xlabel('Time, t [d$_{\mathrm{AE}}$]')
    #plt.title('NP Model of Biomass Growth')

    # Put MEC on top of it
    def mec_model(t, y, crop, CO2, PPF):
        Neq = len(y)

        ## Prepare dydt array
        dydt = np.zeros((1, Neq))

        ## Define dydt
        dydt[0, 0] = calc_m_B(t, crop, CO2, PPF)

        return [np.transpose(dydt)]

    ## Define directory and locations
    directory = 'parameter-lists/'
    filename = 'crop_parameters_FPSD.xlsx'

    ## Load standard parameters from BVAD
    filename_full = directory + filename

    ## Load the standard lettuce crop
    crop_type = 'lettuce'
    crop = Crop(crop_type, filename_full=filename_full)
    crop.t_M = endtime

    ## Perform integration
    start_time = time.time()
    sol_MEC = integrate.solve_ivp(mec_model, tspan, y0, args=(crop,c_CO2,Phi_gamma), method='LSODA', t_eval=t_eval)
   # print("--- %s seconds ---" % (time.time() - start_time))

    ## Convert m_T to m_E (with lettuce t_E = 0)
    sol_MEC.y[0] *= crop.f_E

    #plt.plot(sol_MEC.t, sol_MEC.y[0], linewidth=4, color = 'g', ls='--', label="MEC prediction")
    #plt.legend()
    #plt.show()

    # show function values over time
    f=np.zeros((50,1))
    g=np.zeros((50,1))
    h=np.zeros((50,1))
    j=np.zeros((50,1))

    for i in range(0,50):
        f[i] = calc_mu_N(i, crop, c_CO2, Phi_gamma)
        g[i] = calc_m_N(i, crop, c_CO2, Phi_gamma)
        h[i] = calc_Y_N(i, crop, c_CO2, Phi_gamma)
        j[i] = calc_eta_N(i, crop, c_CO2, Phi_gamma)
    # Create arrays to store m_N and Y_N values for sol_NP.t
    m_N_values = np.zeros(len(sol_NP.t))
    Y_N_values = np.zeros(len(sol_NP.t))

    # Evaluate m_N and Y_N for each time point in sol_NP.t
    for i, t in enumerate(sol_NP.t):
        m_N_values[i] = calc_m_N(t, crop, c_CO2, Phi_gamma)
        Y_N_values[i] = calc_Y_N(t, crop, c_CO2, Phi_gamma)

    # Return the arrays along with the other return values
    return sol_MEC.t, sol_MEC.y[0], sol_NP.t, sol_NP.y[0], m_N_values, Y_N_values
    #plt.plot(np.arange(0,50), f, label="$\mu_N$")
    #plt.plot(np.arange(0,50), g, label="$m_N$")
    #plt.plot(np.arange(0,50), h, label="$\dot{Y}_N$")
    #plt.plot(np.arange(0,50), j, label="$\eta_N$")
    #plt.legend()
    


In [ ]:
def objective_function(x,print_flag):
    """
    Fitting function to be minimized in order to compare
    MEC and NP
    x:Model parameters
    print_flag : whether to plot predictions or not
    """
    t_MEC, y_MEC, t_NP, y_NP, _, _= run_models(x,[])
    if print_flag:
        plt.figure(dpi=300,figsize=(2,2))
        plt.plot(t_MEC, y_MEC, label='MEC Model')
        plt.plot(t_NP, y_NP, label='NP Model')
        plt.legend()
    a = y_MEC
    b = y_NP
    diff = np.abs(a - b) 
    norm = np.linalg.norm(diff, ord=2)
    return norm

In [ ]:
#The nominal parameters - Defined by Kevin
x_nom = np.array([0.0017, 0.225, 7,0.085,38.0, 0.95,0.7,0.01])

In [ ]:
f_ev = objective_function(x_nom,True) #evaluate the error based on nominal parameters
print(f_ev)

## Define Lower/Upper Bounds

In [ ]:
from scipy.optimize import minimize
ls = 0.1 #Multiplier for lower bound
us = 3.0 #Multiplier for upper bound
x0 = [0.0017, 0.225, 7.0 ,0.085,38.0, 0.95,0.7,0.01]  # initial guess for the decision variables
bounds = [(x0[0]*ls,x0[0]*us ), (x0[1]*ls,x0[1]*us ), ((x0[2]*ls,x0[2]*us )),
          (x0[3]*ls,x0[3]*us ),(x0[4]*ls,x0[4]*us ),(x0[5]*ls,x0[5]*us ),
         (x0[6]*ls,x0[6]*us ),(x0[7]*0.8,x0[7]*1.2 ) ]  # bounds on the decision variables

In [ ]:
options = {'disp': False}
result = minimize(objective_function, x0, args=(False) , bounds=bounds,options=options)

In [ ]:
result.x

In [ ]:
f_ev = objective_function(result.x,True) #Re-evaluate based on optimal values 

### Read all experimental data

In [ ]:
import os

data_path = './zea-data/'
for filename in os.listdir(data_path):
    if os.path.isfile(os.path.join(data_path, filename)):
        print(filename)

### Normal Conditions

In [ ]:
import pandas as pd
# Load the CSV file
df = pd.read_csv(data_path + 'zea_biomass-dw-area_normal.csv')
times  = df['dAE']
points = df['mB mean [gDW/m^2]']
# Convert the column to a NumPy array
times_normal = times.values
points_normal = points.values
# Reshape the array to have shape (n, 1)
times_normal =times_normal.reshape(-1, 1)
points_normal = points_normal.reshape(-1,1)

In [ ]:
df = pd.read_csv(data_path + "zea_nitrogen-mass_normal.csv")
times = df['dAE'][0:6]
points = df['mN mean [g]'][0:6]
# Convert the column to a NumPy array
times_N_normal = times.values
points_N_normal = points.values
# Reshape the array to have shape (n, 1)
times_N_normal =times_N_normal.reshape(-1, 1)
points_N_normal = points_N_normal.reshape(-1,1)

In [ ]:
df = pd.read_csv(data_path + "zea_YN_normal.csv")
times = df['dAE'][0:5]
points = df['YN mean [g]'][0:5]
# Convert the column to a NumPy array
times_YN_normal = times.values
points_YN_normal = points.values
# Reshape the array to have shape (n, 1)
times_YN_normal =times_YN_normal.reshape(-1, 1)
points_YN_normal = points_YN_normal.reshape(-1,1)

### Excess Conditions

In [ ]:
df = pd.read_csv(data_path + 'zea_biomass-dw-area_excess.csv')
times  = df['dAE']
points = df['mB mean [gDW/m^2]']
# Convert the column to a NumPy array
times_excess = times.values
points_excess = points.values
# Reshape the array to have shape (n, 1)
times_excess =times_excess.reshape(-1, 1)
points_excess = points_excess.reshape(-1,1)

df = pd.read_csv(data_path + "zea_nitrogen-mass_excess.csv")
times = df['dAE'][0:6]
points = df['mN mean [g]'][0:6]
# Convert the column to a NumPy array
times_N_excess = times.values
points_N_excess = points.values
# Reshape the array to have shape (n, 1)
times_N_excess = times_N_excess.reshape(-1, 1)
points_N_excess = points_N_excess.reshape(-1,1)

df = pd.read_csv(data_path + "zea_YN_excess.csv")
times = df['dAE'][0:5]
points = df['YN mean [g]'][0:5]
# Convert the column to a NumPy array
times_YN_excess = times.values
points_YN_excess = points.values
# Reshape the array to have shape (n, 1)
times_YN_excess = times_YN_excess.reshape(-1, 1)
points_YN_excess = points_YN_excess.reshape(-1,1)


### Deficient

In [ ]:
df = pd.read_csv(data_path + 'zea_biomass-dw-area_deficient.csv')
times  = df['dAE']
points = df['mB mean [gDW/m^2]']
# Convert the column to a NumPy array
times_deficient = times.values
points_deficient = points.values
# Reshape the array to have shape (n, 1)
times_deficient  = times_deficient.reshape(-1, 1)
points_deficient = points_deficient.reshape(-1,1)

########################################################
df = pd.read_csv(data_path + "zea_nitrogen-mass_deficient.csv")
times = df['dAE'][0:6]
points = df['mN mean [g]'][0:6]
# Convert the column to a NumPy array
times_N_deficient = times.values
points_N_deficient = points.values
# Reshape the array to have shape (n, 1)
times_N_deficient = times_N_deficient.reshape(-1, 1)
points_N_deficient = points_N_deficient.reshape(-1,1)
##############################################################

df = pd.read_csv(data_path + "zea_YN_deficient.csv")
times = df['dAE'][0:5]
points = df['YN mean [g]'][0:5]
# Convert the column to a NumPy array
times_YN_deficient = times.values
points_YN_deficient = points.values
# Reshape the array to have shape (n, 1)
times_YN_deficient = times_YN_deficient.reshape(-1, 1)
points_YN_deficient = points_YN_deficient.reshape(-1,1)


In [ ]:
print(times_normal,times_excess,times_deficient)

In [ ]:
print(times_N_normal,times_N_excess,times_N_deficient)

In [ ]:
print(times_YN_normal,times_YN_excess,times_YN_deficient)

In [ ]:
times_normal == times_N_deficient

In [ ]:
plt.figure(figsize=(1.5,1.5))
plt.plot(times_normal, points_normal,'-o')
plt.plot(times_excess, points_excess,'-o')
plt.plot(times_deficient, points_deficient,'-o')

In [ ]:
plt.figure(figsize=(1.5,1.5))
plt.plot(times_N_normal, points_N_normal,'-o')
plt.plot(times_N_excess, points_N_excess,'-o')
plt.plot(times_N_deficient, points_N_deficient,'-o')

In [ ]:
plt.figure(figsize=(1.5,1.5))
plt.plot(times_YN_normal, points_YN_normal,'-o')
plt.plot(times_YN_excess, points_YN_excess,'-o')
plt.plot(times_YN_deficient, points_YN_deficient,'-o')

In [ ]:
def plot_experiment(xnm, xn, xe, xd, print_flag):
    """
    For plotting final predictions after fitting
    xn: parameters for normal fit
    xe: parameters for excess fit
    xd: parameters for deficit fit
    """
    t_eval = np.arange(0, 40, 0.5)
    t_MECnM, y_MECnM, t_NPnM, y_NPnM, mNnm, yNnm = run_models_less(xnm[1:], t_eval)
    
    t_MECn, y_MECn, t_NPn, y_NPn, mNn, yNn = run_models_less(xn, t_eval)
    t_MECe, y_MECe, t_NPe, y_NPe, mNe, yNe = run_models_less(xe, t_eval)
    t_MECd, y_MECd, t_NPd, y_NPd, mNd, yNd = run_models_less(xd, t_eval)
    
    if print_flag:
        # Plotting Biomass Concentration
        # plt.figure(dpi=2000, figsize=(2, 2))
        # plt.plot(t_NPnM, y_NPnM, '--', label='Normal', color=md_color)
        # plt.plot(t_NPn, y_NPn, '-', label='Normal', color=md_color)
        # plt.plot(t_NPe, y_NPe, '-', label='Excess', color=hg_color)
        # plt.plot(t_NPd, y_NPd, '-', label='Deficit', color=lw_color)
        # plt.plot(times_normal, points_normal, 'o', color=md_color)
        # plt.plot(times_excess, points_excess, 'o', color=hg_color)
        # plt.plot(times_deficient, points_deficient, 'o', color=lw_color)
        # plt.xlabel('Time (hr)')
        # plt.ylabel('Biomass Concentration')
        # plt.legend()
        plt.figure(figsize=(2,2), dpi=2000)
        plt.plot(t_NPe, y_NPe, '--',label='NP Excess', color=hg_color)
        plt.plot(t_NPn, y_NPn, '--',label='NP Normal', color=md_color)
        plt.plot(t_NPd, y_NPd, '--',label='NP Deficient', color=lw_color)
        plt.plot(t_NPnM, y_NPnM, '--',label='MEC', color='LimeGreen' )
        #plt.plot(t_MEC, y_MEC, label='MEC Model', color='red')
        plt.plot(times_normal, points_normal,'o',color=md_color)
        plt.plot(times_excess, points_excess,'o',color=hg_color)
        plt.plot(times_deficient, points_deficient,'o',color=lw_color)
        # plt.xlabel("Time, $t\ [\si{\day\of{AE}}]$")
        plt.xlabel("$t\ [\si{\day\of{AE}}]$")
        # plt.ylabel("Areal dry weight biomass,\n $\invbreve{m}_\mathrm{B}\ [\si{\gram\of{DW}\per\meter\squared}]$")
        plt.ylabel("$\invbreve{m}_\mathrm{B}\ [\si{\gram\of{DW}\per\meter\squared}]$")
        plt.gca().xaxis.set_major_locator(MultipleLocator(10))
        plt.gca().xaxis.set_minor_locator(MultipleLocator(5))
        plt.gca().yaxis.set_major_locator(MultipleLocator(20))
        plt.gca().yaxis.set_minor_locator(MultipleLocator(10))
        plt.legend()
        plt.legend(ncol=1, loc="upper left")#,  bbox_to_anchor=(0.5, -0.5))
        plt.savefig(out_path + "lettuce_all_NP-fit.png", bbox_inches='tight', transparent=True)
        
        # Plotting m_N values
        plt.figure(dpi=2000, figsize=(2,2))
        plt.plot(t_NPe, mNe, '--', label='Excess', color=hg_color)
        plt.plot(t_NPn, mNn, '--', label='Normal', color=md_color)
        plt.plot(t_NPd, mNd, '--', label='Deficient', color=lw_color)
        plt.plot(times_N_normal, points_N_normal, 'o', color=md_color)
        plt.plot(times_N_excess, points_N_excess, 'o', color=hg_color)
        plt.plot(times_N_deficient, points_N_deficient, 'o', color=lw_color)
        plt.xlabel("Time, $t\ [\si{\day\of{AE}}]$")
        plt.ylabel('$m_\mathrm{N}\ [\si{\gram}]$')
        plt.gca().xaxis.set_major_locator(MultipleLocator(10))
        plt.gca().xaxis.set_minor_locator(MultipleLocator(5))
        plt.gca().yaxis.set_major_locator(MultipleLocator(0.1))
        plt.gca().yaxis.set_minor_locator(MultipleLocator(0.05))
        plt.legend()
        # plt.title('m_N over Time')
        plt.savefig(out_path + "lettuce_all_mN-fit.png", bbox_inches='tight', transparent=True)
        
        # Plotting Y_N values
        plt.figure(dpi=2000, figsize=(2,2))
        plt.plot(t_NPe, yNe, '--', label='Excess', color=hg_color)
        plt.plot(t_NPn, yNn, '--', label='Normal', color=md_color)
        plt.plot(t_NPd, yNd, '--', label='Deficient', color=lw_color)
        plt.plot(times_YN_normal, points_YN_normal, 'o', color=md_color)
        plt.plot(times_YN_excess, points_YN_excess, 'o', color=hg_color)
        plt.plot(times_YN_deficient, points_YN_deficient, 'o', color=lw_color)
        plt.xlabel("$t\ [\si{\day\of{AE}}]$")
        # plt.ylabel("Areal dry weight biomass,\n $\invbreve{m}_\mathrm{B}\ [\si{\gram\of{DW}\per\meter\squared}]$")
        plt.ylabel("$\dot{Y}_\mathrm{N}\ [\si{\gram\of{DW}\per\day\per\gram\of{N}}]$")
        plt.gca().xaxis.set_major_locator(MultipleLocator(10))
        plt.gca().xaxis.set_minor_locator(MultipleLocator(5))
        plt.gca().yaxis.set_major_locator(MultipleLocator(1))
        plt.gca().yaxis.set_minor_locator(MultipleLocator(0.5))
        plt.legend()
        # plt.title('Y_N over Time')
        plt.savefig(out_path + "lettuce_all_YN-fit.png", bbox_inches='tight', transparent=True)
    
    return None


In [ ]:
def objective_experiment(x, print_flag, condition):
    
    # Extract m_N, Y_N, and biomass values from the model
    t_MEC, y_MEC, t_NP, y_NP, m_N, Y_N = run_models_less(x, times_normal)
    
    # Define the experimental data for m_N, Y_N, and biomass based on the condition
    if condition == 'Normal':
        mB_exp = points_normal
        mN_exp = points_N_normal
        YN_exp = points_YN_normal
    elif condition == 'Excess':
        mB_exp = points_excess
        mN_exp = points_N_excess
        YN_exp = points_YN_excess
    elif condition == 'Deficit':
        mB_exp = points_deficient
        mN_exp = points_N_deficient
        YN_exp = points_YN_deficient
    
    w = 2.0
    
    # Compute relative errors for m_N, Y_N, and biomass
    rel_diff_mB = w*np.abs(mB_exp - y_NP.reshape(-1, 1)) / mB_exp
    rel_diff_mN = np.abs(mN_exp - m_N.reshape(-1, 1)) / mN_exp
    rel_diff_YN = np.abs(YN_exp -  Y_N[1:].reshape(-1, 1)) / YN_exp
    
    # Combine the relative errors
    diff_vec = np.concatenate((rel_diff_mB, rel_diff_mN, rel_diff_YN), axis=0)
    
    # Compute the overall objective
    norm = np.linalg.norm(diff_vec, ord=2)
    
    # Optional plotting
    if print_flag:
        # colors = {'Normal': 'red', 'Excess': 'blue', 'Deficit': 'green'}
        colors = {'Normal': 'DodgerBlue', 'Excess': 'MidnightBlue', 'Deficit': 'SkyBlue'}
        
        # Plotting Biomass
        plt.figure(dpi=300, figsize=(2, 2))
        # plt.plot(t_NP, y_NP, '--', label='NP Model', color=colors[condition])
        plt.plot(t_NP, y_NP, '--', label='NP Model', color=np_color)
        plt.plot(times_normal, points_normal, 'o', color=md_color)
        plt.plot(times_excess, points_excess, 'o', color=hg_color)
        plt.plot(times_deficient, points_deficient, 'o', color=lw_color)
        plt.xlabel('Time (hr)')
        plt.ylabel('Biomass Concentration')
        plt.legend()
        
        # Plotting m_N
        plt.figure(dpi=300, figsize=(2, 2))
        plt.plot(t_NP, m_N, '--', label='m_N Model', color=colors[condition])
        plt.plot(times_N_normal, points_N_normal, 'o', color=md_color)
        plt.plot(times_N_excess, points_N_excess, 'o', color=hg_color)
        plt.plot(times_N_deficient, points_N_deficient, 'o', color=lw_color)
        plt.xlabel('Time (hr)')
        plt.ylabel('m_N Concentration')
        plt.legend()
        
        # Plotting Y_N
        plt.figure(dpi=300, figsize=(2, 2))
        plt.plot(t_NP[1:], Y_N[1:], '--', label='Y_N Model', color=colors[condition])
        plt.plot(times_YN_normal, points_YN_normal, 'o', color=md_color)
        plt.plot(times_YN_excess, points_YN_excess, 'o', color=hg_color)
        plt.plot(times_YN_deficient, points_YN_deficient, 'o', color=lw_color)
        plt.xlabel('Time (hr)')
        plt.ylabel('Y_N Value')
        plt.legend()
    
    return norm

In [ ]:
objective_experiment(result.x[1:], True,'Normal')

### Here starts the fitting to data: Initial guess is the fit we did before

In [ ]:
from scipy.optimize import minimize
from scipy.optimize import basinhopping
from scipy.optimize import differential_evolution

ls = 0.01
us = 10.0
x0 = result.x[1:] #Start from 1 since we fix mN0
#x0 = [0.0017, 0.225, 7.0 ,0.085,38.0, 0.95,0.7]  # initial guess for the decision variables
bounds = [(x0[0]*ls,x0[0]*us ), (x0[1]*ls,x0[1]*us ), ((x0[2]*ls,x0[2]*us )),
          (x0[3]*ls,x0[3]*us ),(x0[4]*ls,x0[4]*us ),(x0[5]*ls,x0[5]*us ),
         (x0[6]*0.8,x0[6]*1.2 )]  # bounds on the decision variables
#options = {'disp': True}
#result_normal = minimize(objective_experiment, x0,  method='TNC',args=(False,'Normal') , bounds=bounds, options=options)
result_normal = differential_evolution(objective_experiment, bounds, args=(False, 'Normal'), maxiter=1000, popsize=15, disp=True)
objective_experiment(result_normal.x,True,"Normal")

In [ ]:
from scipy.optimize import minimize
#ls = 0.01
#us = 10.0
ls = 0.2
us = 5.0
#x0 = result.x[1:] #Start from 1 since we fix mN0
x0 = result_normal.x
#x0 = [0.0017, 0.225, 7.0 ,0.085,38.0, 0.95,0.7]  # initial guess for the decision variables
bounds = [(x0[0]*ls,x0[0]*us ), (x0[1]*ls,x0[1]*us ), ((x0[2]*ls,x0[2]*us )),
          (x0[3]*ls,x0[3]*us ),(x0[4]*ls,x0[4]*us ),(x0[5]*ls,x0[5]*us ),
         (x0[6]*0.8,x0[6]*1.3 )]  # bounds on the decision variables
options = {'disp': True}
#result_excess = minimize(objective_experiment, x0, args=(False,'Excess') , bounds=bounds, options=options)
result_excess = differential_evolution(objective_experiment, bounds, args=(False, 'Excess'), maxiter=1000, popsize=15, disp=True)

objective_experiment(result_excess.x,True,"Excess")

In [ ]:
from scipy.optimize import minimize
#ls = 0.01
#us = 10.0
ls = 0.2
us = 5.0
#x0 = result.x[1:] #Start from 1 since we fix mN0
x0 = result_normal.x
#x0 = [0.0017, 0.225, 7.0 ,0.085,38.0, 0.95,0.7]  # initial guess for the decision variables
bounds = [(x0[0]*ls,x0[0]*us ), (x0[1]*ls,x0[1]*us ), ((x0[2]*ls,x0[2]*us )),
          (x0[3]*ls,x0[3]*us ),(x0[4]*ls,x0[4]*us ),(x0[5]*ls,x0[5]*us ),
         (x0[6]*0.8,x0[6]*1.3 )]  # bounds on the decision variables
options = {'disp': True}
#result_excess = minimize(objective_experiment, x0, args=(False,'Excess') , bounds=bounds, options=options)
result_deficit = differential_evolution(objective_experiment, bounds, args=(False, 'Deficit'), maxiter=1000, popsize=15, disp=True)

objective_experiment(result_deficit.x,True,"Deficit")

In [ ]:
plot_experiment(result.x, result_normal.x,result_excess.x,result_deficit.x,True)
plt.show()
#Same colors as before, dashed line is MEC model for normal

### Now, let's gather all variables into a matrix

In [ ]:
result_deficit.x

In [ ]:
result_excess.x

In [ ]:
result_normal.x

In [ ]:
result.x[1:]

In [ ]:
all_params = np.column_stack((result.x[1:], result_normal.x, result_excess.x, result_deficit.x))

In [ ]:
all_params.shape

In [ ]:
all_params

In [ ]:
#Extracting the min and max value from the optimization variables to 
#get bounds for sensitivities
min_values = np.min(all_params, axis=1)
max_values = np.max(all_params, axis=1)

In [ ]:
bounds = np.column_stack((min_values, max_values))

In [ ]:
bounds.shape[0]

In [ ]:
# Generate random samples of x within the bounds
num_samples = 2000
x = np.zeros((num_samples,bounds.shape[0]))
for i in range(bounds.shape[0]):
    x[:, i] = np.random.uniform(low=bounds[i, 0], high=bounds[i, 1], size=num_samples)

In [ ]:
x.shape
f_vals = np.zeros((num_samples,1))

In [ ]:
# Define the function that yields the quantitity whose sensitivity we're 
# studying

In [ ]:
def f_sens(x,k):
    """
    For each parameter x, 
    get an integral of the curve over time
    This is a representative scalar quantity
    that reflects how the parameters x affect the dynamics 
    in an average sense
    """
    tspan = np.arange(0, 30.05, 0.05)
    t_MEC, y_MEC, t_NP, y_NP, _, _ = run_models_less(x,tspan)
    integral = np.trapz(y_NP, t_NP)
    #plt.plot(t_NP,y_NP,'-',alpha=k/10,linewidth=5)
    return integral   

In [ ]:
#f_vals = np.zeros((num_samples,1))
for k in range(num_samples):
    #print(x[k,:])
    f_vals[k,0] = f_sens(x[k,:],k)
    #print(f_vals[k,0])

In [ ]:
f_vals.shape

In [ ]:
np.median(f_vals)
benchmark_val = 0.5*np.median(f_vals) 

In [ ]:
f_vals_modified = f_vals - benchmark_val

### Issue: A lot of parameter combinations lead to unphysical predictions
To help generate only physical predictions, we can build a classifier that 
can predict whether the prediction will be >0 or <0

In [ ]:
from tensorflow.keras.utils import to_categorical
Yc = np.sign(f_vals_modified);
y_binary = np.where(Yc==1,1,0)
x_binary = x;

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x_binary, y_binary, test_size=0.2, random_state=42)

In [ ]:
Y_train

In [ ]:
sum(Y_train)

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.python.framework import ops
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
x0.shape

In [ ]:
model = Sequential()
model.add(Dense(64, input_dim=x0.shape[0], activation='tanh'))
model.add(Dense(32, activation='tanh'))
model.add(Dense(32, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))

# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit model
model.fit(X_train, Y_train, epochs=1000, batch_size=32, verbose=1)

In [ ]:
# predict Y for new X
predicted_Y = np.round(model.predict(X_test))

In [ ]:
#Find error of classifier 
from sklearn.metrics import accuracy_score, f1_score
# Y_test and predicted_Y are the actual and predicted outputs, respectively
acc = accuracy_score(Y_test, predicted_Y)
f1 = f1_score(Y_test, predicted_Y)
print("Accuracy:", acc)
print("F1 score:", f1)

In [ ]:
#Now that we have the classifier, let's get good samples for SA.
# Generate new random samples of x within the bounds
num_samples = int(10e5)
xs = np.zeros((num_samples,bounds.shape[0]))
for i in range(bounds.shape[0]):
    xs[:, i] = np.random.uniform(low=bounds[i, 0], high=bounds[i, 1], size=num_samples)

In [ ]:
bounds

In [ ]:
#Let's pass them through the classifier and predict whether they will yield 
#a physically meaningful or not prediction. 
# We want all classifier values with 1 (which means positive)
predicted_ys =model.predict(xs)

In [ ]:
predicted_ys

In [ ]:
positive_indices = [i for i in range(len(predicted_ys)) if predicted_ys[i] > 0.5]

In [ ]:
len(positive_indices)/num_samples

In [ ]:
len(positive_indices)

In [ ]:
sobol_samples = len(positive_indices)

In [ ]:
Nsobol = 20000 #Sobol analysis samples - must be at max equal to 'sobol_samples'

In [ ]:
xsobol = xs[positive_indices]

In [ ]:
xsobol = xsobol[0:Nsobol]

In [ ]:
xsobol

In [ ]:
fsobol = np.zeros((Nsobol,1))

In [ ]:
Nsobol

In [ ]:
for k in range(Nsobol):
    #print(x[k,:])
    fsobol[k,0] = f_sens(xsobol[k,:],k)
    #print(f_vals[k,0])

In [ ]:
# But we're not done, due to inaccuracy of the classifier not all predicted 
# integrals will be positive. We need to keep only the positive.
# The classifier helped get samples that are more likely to give positive 
# values.

In [ ]:
min(fsobol)

In [ ]:
medfv = np.median(f_vals)

In [ ]:
positive_sobol = np.where(fsobol > medfv )[0]
xsobol_adjusted = xsobol[positive_sobol]
fsobol_adjusted = fsobol[positive_sobol]

In [ ]:
positive_sobol.shape

In [ ]:
import scipy
import scipy.io as  sio
sio.savemat(mat_out_path + 'lettuce_exp_matrices.mat', {'matrix1': xsobol_adjusted, 'matrix2': fsobol_adjusted})

## Stop here and run the sensitivity analysis on matlab

In [ ]:
sens_indices = scipy.io.loadmat(mat_out_path + 'lettuce_exp_sensitivity_results.mat')

In [ ]:
sensitivity_indices = sens_indices['sensitivities']

In [ ]:
# # Create a list of labels for the plot
# labels = ['$x_0$', '$x_1$', '$x_2$', '$x_3$', '$x_4$', '$x_5$', '$x_6$']

# # Create the bar plot with black border
# fig, ax = plt.subplots(dpi=500)
# ax.barh(labels, sensitivity_indices[:, 0], edgecolor='black')

# # Add labels to the plot
# ax.set_xlabel('Sensitivity Index Value')
# #ax.set_title('Sensitivity Indices')
# ax.set_ylabel('Variable')
# # Show the plot
# plt.show()

# Create a list of labels for the plot
# labels = ['$x_0$', '$x_1$', '$x_2$', '$x_3$', '$x_4$', '$x_5$', '$x_6$']
labels = ['$r$', '$K$', r'$\alpha$', r'$\eta_\text{N} (b)$', r'$\eta_\text{N} (m)$', r'$\mu_\text{N} (b)$', r'$\mu_\text{N} (m)$']

# Create the bar plot with black border
fig, ax = plt.subplots(figsize=(2,2))
ax.barh(labels, sensitivity_indices[:, 0], edgecolor='black')

# Add labels to the plot
ax.set_xlabel('Sensitivity Index Value')
#ax.set_title('Sensitivity Indices')
ax.set_ylabel('Parameter')

ax.xaxis.set_major_locator(MultipleLocator(0.05))
ax.xaxis.set_minor_locator(MultipleLocator(0.025))

plt.gca().invert_yaxis()

plt.savefig(out_path + "lettuce_exp-sensitivity.png", bbox_inches='tight', transparent=True)

# Show the plot
plt.show()

## Generate and plot hybrid MEC-NP

In [ ]:
t_end = 30
t_length = np.arange(0, t_end, 1)

df = pd.DataFrame()
df["regime"] = ["lw","md","hi"]

# t_MECn, y_MECn, t_NPn, y_NPn = run_models_less(result_normal.x,np.arange(0, t_end+1, 1))
# t_MECe, y_MECe, t_NPe, y_NPe = run_models_less(result_excess.x,np.arange(0, t_end+1, 1))

result_lw = run_models_less((result_deficit.x),t_length)
result_md = run_models_less((result_normal.x),t_length)
result_hi = run_models_less((result_excess.x),t_length)

df["tMEC"] = [result_lw[0], result_md[0], result_hi[0]]
df["yMEC"] = [result_lw[1], result_md[1], result_hi[1]]
df["tNP"] = [result_lw[2], result_md[2], result_hi[2]]
df["yNP"] = [result_lw[3], result_md[3], result_hi[3]]
df["yNP"] *= 1.5

df2 = pd.DataFrame()
df2["cond"] = ["lw","md","hi"]
df2["time"] = [result_lw[0]+1, result_md[0]+1, result_hi[0]+1]
df2["hybrid_curve"] = [[],[],[]]

for regime in range(0,3):
    hyb_crv = [0 for i in range(0,t_end)]
    for t in range(0,t_end):
        if df.iloc[1,2][t] > df.iloc[regime,4][t]:  # we have only to MEC-Normal
            hyb_crv[t] = df.iloc[regime,4][t]
        else:
            hyb_crv[t] = df.iloc[regime,2][t]
    df2.at[regime, "hybrid_curve"] = hyb_crv
    
clr_hybrid = ['k','k','k']
lbl_hybrid = ["Deficienct","Normal","Excess"]
fig_hybrid, ax_hybrid = plt.subplots(figsize=(1.5,2.625))
hybird_args = {'alpha': 1, 'lw': 2.5, 'dashes':[2.5, 1]}
regime = 1  # {0,1,2} |-> {low, med, high}

ax_hybrid.plot(df2.iloc[regime,1],df2.iloc[regime,2],c="red", label="Hybrid", zorder=99, alpha=1, lw=1)
ax_hybrid.plot(df.iloc[regime,1]+1,df.iloc[regime,2],c='LimeGreen', ls="--", label="MEC", **hybird_args)
ax_hybrid.plot(df.iloc[regime,3]+1,df.iloc[regime,4],c='DodgerBlue', ls="--", label="NP", **hybird_args)

from matplotlib.ticker import (MultipleLocator)
ax_hybrid.xaxis.set_major_locator(MultipleLocator(10))
ax_hybrid.xaxis.set_minor_locator(MultipleLocator(5))
ax_hybrid.yaxis.set_major_locator(MultipleLocator(20))
ax_hybrid.yaxis.set_minor_locator(MultipleLocator(10))
ax_hybrid.set_xlabel(r'$t [\si{\day\of{AE}}]$')
ax_hybrid.set_ylabel(r'$\invbreve{m}_\text{T}\ [\si{\gram\of{DW}\per\meter\squared}]$')

# y0 = df.iloc[regime,4]
# y1 = df.iloc[regime,2]
# ax_hybrid.fill_between(t_length+1,y0,y1,color='pink',alpha=0.5)

plt.legend()
plt.savefig(out_path + "MEC-NP_hybrid.png", bbox_inches='tight', transparent=True, dpi=2000)
plt.show()

## Draft Code Below this point
I tried to compare the prior/posterior distributions but nothing interesting really

In [ ]:
xsobol_adjusted.shape[0]

In [ ]:
bounds.shape

In [ ]:
N = xsobol_adjusted.shape[0]
# Generate N samples for each variable
samples = np.zeros((N, 7))
for i in range(7):
    samples[:, i] = np.random.uniform(bounds[i, 0], bounds[i, 1], size=N)

In [ ]:
samples.shape

In [ ]:
xsobol_adjusted.shape

In [ ]:
binnum=50
for k in range(samples.shape[1]):
    plt.figure(dpi=100,figsize=(2,2))
    plt.hist(samples[:,k],bins=binnum,alpha=0.6,color='red',density=True)

    plt.hist(xsobol_adjusted[:,k],bins=binnum,alpha=0.5, color=hg_color,density=True)

    

In [ ]:
bounds

In [ ]:
# calculate the range of each variable
range_ = bounds[:, 1] - bounds[:, 0]

# calculate the mean value of each variable
mean_ = np.mean(bounds, axis=1)

# calculate the upper bound value of each variable
upper_ = bounds[:, 1]

# normalize the range by the mean value
relative_change_mean = range_ / mean_

# normalize the range by the upper bound value
relative_change_upper = range_ / upper_

# print the normalized range
for i in range(len(relative_change_mean)):
    print(f"Variable {i+1}: Relative change (mean) = {relative_change_mean[i]:.2%}, Relative change (upper) = {relative_change_upper[i]:.2%}")